In [1]:
import arcpy
import requests
import json
import datetime
import os
import io
import zipfile
import shapefile
from shapely.geometry import Point, shape
from shapely import geometry
import numpy as np
import psycopg2
import pandas as pd
import fiona
from fiona.crs import from_epsg
from osgeo import gdal

In [2]:
# Set workspace
os.chdir(r'E:\ArcGIS_2\Lab3')
wksp = os.getcwd()

# 1. TEMPERATURE

### Pull weather information

In [3]:
# Get the 153 weather stations of the Minnesota network selecting a random day
link = r'https://mesonet.agron.iastate.edu/api/1/daily.geojson?date=2023-03-15&network=MN_RWIS'
info = json.loads(requests.get(link).text)
location = []
for i in range(len(info['features'])):
    # Store the station and its coordinates
    location.append({'station': info['features'][i]['properties']['station'], 
                     'coordinates': info['features'][i]['geometry']['coordinates']})

In [4]:
# Daily weather data from 2023
url = r'https://mesonet.agron.iastate.edu/api/1/daily.geojson?network=MN_RWIS&year=2023'
weather = json.loads(requests.get(url).text)

In [5]:
# Delete all the unneeded weather variables and keep only minimum and maximum temperature
delete = [
      "tmpf_est",
      "precip",
      "precip_est",
      "max_gust",
      "snow",
      "snowd",
      "min_rh",
      "max_rh",
      "min_dwpf",
      "max_dwpf",
      "min_feel",
      "max_feel",
      "min_rstage",
      "max_rstage",
      "temp_hour",
      "max_gust_localts",
      "max_drct",
      "avg_feel", 
      "avg_sknt", 
      "vector_avg_drct", 
      "id"
]

for i in range(len(weather['features'])):
    for key in delete:
        del weather['features'][i]['properties'][key]

### Pull Minnesota boundary from MGC

In [6]:
# MN boundary
mn_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/bdry_state/shp_bdry_state.zip"
boundaries = requests.post(mn_url)
zipfile.ZipFile(io.BytesIO(boundaries.content)).extractall(wksp)

# Project shp
sr = arcpy.SpatialReference(4326)
arcpy.Project_management('Boundaries_of_Minnesota.shp', 'minnesota.shp', sr)

<Result 'E:\\ArcGIS_2\\Lab3\\minnesota.shp'>

### QA

In [7]:
# Read in the shapefile data for Minnesota
sf = shapefile.Reader("minnesota.shp")
shapes = sf.shapes()
state_border = shapes[0]

# Create a shapely Polygon object from the state border shape
border_polygon = shape(state_border)

# Create an empty list to add the not useful readings
wrong = []

# Add all the not useful readings to a list
for i in range(len(weather['features'])):
    
    # Readings whose temp readings are None
    if weather['features'][i]['properties']['min_tmpf'] == None or weather['features'][i]['properties']['max_tmpf'] == None:
        wrong.append(weather['features'][i])
        continue
    
    # Stations outside of Minnesota
    point = Point(weather['features'][i]['geometry']['coordinates'])
    if not border_polygon.contains(point):
        wrong.append(weather['features'][i])
        continue
        
    # Readings whose min and max temp are the same. This is an indicator of wrong data
    if weather['features'][i]['properties']['min_tmpf'] == weather['features'][i]['properties']['max_tmpf']:
        wrong.append(weather['features'][i])
        continue
        
    # Readings whose temp is outside of the optimum range
    lower_limit = math.floor(weather['features'][i]['properties']['min_tmpf'])
    upper_limit = math.ceil(weather['features'][i]['properties']['max_tmpf'])
    range_temp = range(lower_limit, upper_limit)
    
    # Readings not representative of the broader region if max and min temp are similar
    if len(range_temp) == 1:
        wrong.append(weather['features'][i]) 
        continue
    
# Delete the not useful readings 
for element in wrong:
    weather['features'].remove(element)

### Monthly average temperature

In [8]:
stations = []
# Add the dictionaries to a data frame
for j in range(len(weather['features'])):
    stations.append(weather['features'][j]['properties'])
df = pd.DataFrame.from_dict(stations)

# Remove the day part from the date leaving only year and month
for i in range(len(df['date'])):
    df['date'][i] = df['date'][i][:7]
    
# Get monthly average min and max temperature for each station
grouped = df.groupby(['station', 'date', 'name']).agg('mean')
grouped.reset_index(inplace=True)

# Return data to a dictionary
mean = grouped.to_dict('records')

[8]:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
## Add the geometry to the stations
mean_tmp = []
for i in range(len(mean)):
    for j in range(len(location)):
        if mean[i]['station'] == location[j]['station']:
            mean_tmp.append({'type': 'Feature', 'properties': mean[i], 
                             'geometry': {'type': 'Point', 'coordinates': location[j]['coordinates']}})

### Stations shapefile

In [11]:
schema =  {'geometry': 'Point', 'properties': {'station': 'str', 'date': 'str', 'name': 'str', 'max_tmpf': 'float', 'min_tmpf': 'float'}}

with fiona.open("stations.shp", 'w', crs = from_epsg(4326), driver = 'ESRI Shapefile', schema = schema) as output:
    for i in range(len(mean_tmp)):
          # geometry
          point = Point(mean_tmp[i]['geometry']['coordinates'])
          # attributes
          prop = mean_tmp[i]['properties']
          # write the row (geometry + attributes in GeoJSON format)
          output.write({'geometry': geometry.mapping(point), 'properties':prop})

# SAVE TO POSTGIS

In [12]:
# Connect to postgresql database
connection = psycopg2.connect(host = '34.27.219.64',
                              port = '5432',
                              database = 'lab1',
                              user = 'postgres',
                              password = 'student',
                             )

In [13]:
# Path and fields of the data to load to the database
data = os.path.join(wksp, "stations.shp")
fields = ["station", "date", "name", "max_tmpf", "min_tmpf", "Shape@WKT"]

# Create SQL table
cursor = connection.cursor()
cursor.execute("DROP TABLE IF EXISTS stations")
cursor.execute("""
    CREATE TABLE stations (
        id SERIAL,
        station VARCHAR,
        date VARCHAR,
        name VARCHAR,
        max_tmpf DOUBLE PRECISION,
        min_tmpf DOUBLE PRECISION)
""")

cursor.execute("""
    SELECT AddGeometryColumn('stations', 'geom', 4326, 'POINT', 2)
""")

# Populate table
with arcpy.da.SearchCursor(data, fields) as da_cursor:
    for row in da_cursor:
        wkt = row[5]
        cursor.execute("INSERT INTO stations (station, date, name, max_tmpf, min_tmpf, geom) VALUES (%s, %s, %s, %s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], row[2], row[3], row[4], wkt))

connection.commit()